In [ ]:
from models.nested_unet import nested_unet
from models.attention_unet import attention_unet
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
import numpy as np


In [ ]:
# Load preprocessed data
train_images = np.load('data/processed/train_images.npy')
test_images = np.load('data/processed/test_images.npy')
train_masks = np.load('data/processed/train_masks.npy')
test_masks = np.load('data/processed/test_masks.npy')





In [ ]:
# Define DICE coefficient metric
def dice_coefficient(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + 1e-7) / (K.sum(y_true_f) + K.sum(y_pred_f) + 1e-7)


In [ ]:
# Instantiate both models
input_shape = (256, 256, 1)
model_nested_unet = nested_unet(input_shape)
model_attention_unet = attention_unet(input_shape)




In [ ]:
# Compile models
model_nested_unet.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=[dice_coefficient])
model_attention_unet.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=[dice_coefficient])

# Train models
model_nested_unet.fit(train_images, train_masks, validation_data=(test_images, test_masks), epochs=50)
model_attention_unet.fit(train_images, train_masks, validation_data=(test_images, test_masks), epochs=50)

# Save trained models
model_nested_unet.save('models/weights/nested_unet.h5')
model_attention_unet.save('models/weights/attention_unet.h5')